# Training supervised
In this notebook I will train a simple supervised MNIST model, so that we have comparison for our model trained with the GAN. The model is based on https://github.com/pfnet/chainer/blob/master/examples/mnist/train_mnist.py i only did some small adaptions to make it more similar to the GAN example.

In [2]:
from __future__ import print_function
import argparse
import sys,os
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

This the network used. I added one hidden layer and set the number of hidden units to the number used by the GAN. Furthermore I changed the relu activation function to the elu function as this is also the one used by the GAN.

In [3]:
# Network definition
class MLP(chainer.Chain):

    def __init__(self, n_out):
        super(MLP, self).__init__(
            # the size of the inputs to each layer will be inferred
            l1=L.Linear(None, 1000),  # n_in -> n_units
            l2=L.Linear(None, 500),  # n_units -> n_units
            l3=L.Linear(None, 250),  # n_units -> n_units
            l4=L.Linear(None, n_out),  # n_units -> n_out
        )

    def __call__(self, x):
        h1 = F.elu(self.l1(x))
        h2 = F.elu(self.l2(h1))
        h3 = F.elu(self.l3(h2))
        return self.l4(h3)

We can largely re-use the batch script from the tutorial. I overwrite arguments variable such that argparse works correctly. I also make sure the model it self is serialized, not only the complete trainer. This makes life a little easier once we want to load the model in order to create adversarial images.

In [4]:
GPU = -1
sys.argv=['main','-b','128','-g',str(GPU)]

In [5]:
def main():
    parser = argparse.ArgumentParser(description='Chainer example: MNIST')
    parser.add_argument('--batchsize', '-b', type=int, default=100,
                        help='Number of images in each mini-batch')
    parser.add_argument('--epoch', '-e', type=int, default=20,
                        help='Number of sweeps over the dataset to train')
    parser.add_argument('--gpu', '-g', type=int, default=-1,
                        help='GPU ID (negative value indicates CPU)')
    parser.add_argument('--out', '-o', default='result',
                        help='Directory to output the result')
    parser.add_argument('--resume', '-r', default='',
                        help='Resume the training from snapshot')
    args = parser.parse_args()

    print('GPU: {}'.format(args.gpu))
    print('# Minibatch-size: {}'.format(args.batchsize))
    print('# epoch: {}'.format(args.epoch))
    print('')

    # Set up a neural network to train
    # Classifier reports softmax cross entropy loss and accuracy at every
    # iteration, which will be used by the PrintReport extension below.
    model = L.Classifier(MLP( 10))
    if args.gpu >= 0:
        chainer.cuda.get_device(args.gpu).use()  # Make a specified GPU current
        model.to_gpu()  # Copy the model to the GPU

    # Setup an optimizer
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    # Load the MNIST dataset
    train, test = chainer.datasets.get_mnist()

    train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
    test_iter = chainer.iterators.SerialIterator(test, args.batchsize,
                                                 repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.StandardUpdater(train_iter, optimizer, device=args.gpu)
    trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)

    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=args.gpu))

    # Dump a computational graph from 'loss' variable at the first iteration
    # The "main" refers to the target link of the "main" optimizer.
    trainer.extend(extensions.dump_graph('main/loss'))

    # Take a snapshot at each epoch
    trainer.extend(extensions.snapshot(), trigger=(args.epoch, 'epoch'))

    # Write a log of evaluation statistics for each epoch
    trainer.extend(extensions.LogReport())

    # Print selected entries of the log to stdout
    # Here "main" refers to the target link of the "main" optimizer again, and
    # "validation" refers to the default name of the Evaluator extension.
    # Entries other than 'epoch' are reported by the Classifier link, called by
    # either the updater or the evaluator.
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    # Print a progress bar to stdout
    trainer.extend(extensions.ProgressBar())

    if args.resume:
        # Resume from a snapshot
        chainer.serializers.load_npz(args.resume, trainer)

    # Run the training
    trainer.run()
    
    #export model so we can load it in our model without having to load the full trainer
    chainer.serializers.save_npz(os.path.join(args.out,'model.npz'),model)

In [6]:
main()

GPU: -1
# Minibatch-size: 128
# epoch: 20

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [..................................................]  1.07%
this epoch [##########........................................] 21.33%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [#.................................................]  2.13%
this epoch [#####################.............................] 42.67%
       200 iter, 0 epoch / 20 epochs
    26.321 iters/sec. Estimated time to finish: 0:05:48.577524.
     total [#.................................................]  3.20%
this epoch [################################..................] 64.00%
       300 iter, 0 epoch / 20 epochs
     26.95 iters/sec. Estimated time to finish: 0:05:36.735091.
     total [##................................................]  4.27%
this epoch [##########################################........] 